# Estimando lucro de regiões petrolíferas

Este projeto tem o objetivo de escolher a melhor região para prospectar petróleo com base no lucro potencial de exploração da região. Os dados disponíveis são da empresa *fictícia* Oleobrás, que disponibilizou as características e o volume de petróleo produzido pelos poços (em mil barris) em três regiões diferentes. 

Para obter a melhor escolha, o projeto primeiro ajusta e utiliza modelos de aprendizado de máquina para estimar a produção de petróleo das regiões. Em seguida, o projeto estimata o lucro de exploração das regiões, considerando os gastos de prospecção. 
Por fim, o projeto disponibiliza recomendações de qual região deve ser explorada com base nos valores e incertezas das estimativcas.

## Importando bibliotecas e dados

In [1]:
### importando bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt

In [2]:
### importando dados
geo0 = pd.read_csv('/datasets/geo_data_0.csv')
geo1 = pd.read_csv('/datasets/geo_data_1.csv')
geo2 = pd.read_csv('/datasets/geo_data_2.csv')

## Processando dados

### Estudando dados

In [3]:
geo0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
geo1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
geo2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


**Comentário**: Todos os conjuntos de dados completos, sem dados faltantes.

In [6]:
geo0.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [7]:
geo1.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [8]:
geo2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


**Comentário**: As características (`f0`, `f1`, `f2`) possuem uma escala de variação e amplitude diferentes. Logo, aplicarei a padronização das variáveis. 

### Tratando dados

In [9]:
### definindo função para padronizar variáveis
def standardize(df):
    df_std= (df-df.mean())/df.std()
    return df_std

In [10]:
### nome das variáveis para serem padronizadas
feature_colum_name = ["f0", "f1", "f2"]

In [11]:
### padronizando as variáveis
geo0[feature_colum_name] = standardize(geo0[feature_colum_name])
geo1[feature_colum_name] = standardize(geo1[feature_colum_name])
geo2[feature_colum_name] = standardize(geo2[feature_colum_name])

**Comentário**: Padronizei todas as variáveis de predição (características) para evitar que as diferenças de escala interfiram no treinamento.

## Avaliando produção de petróleo das regiões

### Definindo função para prever a produção de petróleo

In [12]:
def geo_evaluation(geo):
    ## separando objetivos e características
    target = geo['product']
    features = geo.drop(['id','product'],  axis = 1)
    ## dividindo em treino e validação
    target_train, target_valid, features_train, features_valid = train_test_split(target, features,
                                                                 train_size = 0.75, random_state = 42)
    ## ajustando modelo
    lm = LinearRegression().fit(features_train, target_train)
    ## predizendo
    prediction = lm.predict(features_valid)
    mean_predict = np.mean(prediction)
    ## avaliando predição 
    rmse = np.sqrt( mean_squared_error(target_valid, prediction) )
    ## exportando resultados
    return mean_predict, rmse

**Comentário**: Defini uma função para avaliação da produção de petróleo em cada região. Para isso a função:
- separa as características e objetivo;
- divide as características e objetivo em 75% treino e 25% validação;
- ajusta um modelo linear com os dados de treino;
- prediz a produção de petróleo com as características de validação;
- calcula o valor médio de produção previsto pelo modelo;
- calcula o erro médio do modelo com base na raiz quadrada do erro quadrático médio (REQM);

### Aplicando a função a diferentes regiões

In [13]:
### oraganizando todos os dados a serem avaliados
all_geos = [geo0, geo1, geo2]
region = ["geo0", "geo1", "geo2"]

In [14]:
### col
predicted_product = []
mean_error =[]
for one_geo in all_geos:
    mean_predict, rmse = geo_evaluation(geo = one_geo)
    predicted_product.append(mean_predict)
    mean_error.append(rmse)

In [15]:
model_eval = pd.DataFrame({"region": region, "mean_predicted_product": predicted_product, "mean_error": mean_error})
model_eval

,region,mean_predicted_product,mean_error
0,geo0,92.398800,37.756600
1,geo1,68.712878,0.890280
2,geo2,94.771024,40.145872


**Comentário**: Baseado nos modelos de regressão, as regiões 0 (`geo0`) e 2 (`geo2`) tem a maior produção de petróleo predita, mas com erros maiores sobre as predições. Em contraste, a região 1 (`geo1`) tem a menor produção de petróleo predita, mas com o menor erro sobre as predições.

### Limiar da produção lucrativa

In [7]:
### preço de mil barris de petróleo
product_price = 4500
### custo de prospectar 200 poços
cost = 100000000
### razão entre custo e preço unitário
rate_cost_price = round(cost/product_price)
print(f"O custo de prospecção de 200 poços é {rate_cost_price} vezes maior do que a produção de mil barris de petróleo")

O custo de prospecção de 200 poços é 22222 vezes maior do que a produção de mil barris de petróleo


In [10]:
### relação custo e benefício
production_per_well = round(rate_cost_price / 200)
print(f"Cada poço deve produzir mais que {production_per_well} mil barris para superar os custos de prospecção")

Cada poço deve produzir mais que 111 mil barris para superar os custos de prospecção


**Comentário**: A produção mínima lucrativa de um poço deve ser 111 mil barris. Considerando este limiar, todas as regiões tem uma produção média por poço abaixo do limiar do lucro (gerariam prejuízos). Contudo, como a produção média das regiões é uma estimativa com erro, é necessário ver a distribuição do lucro das regiões.

## Projetando lucro

### Definindo funções para estimar e decrever o lucro líquido

In [21]:
def project_revenue(geo):
    ## separando objetivos e características
    target = geo['product']
    features = geo.drop(['id','product'],  axis = 1)
    ## vetores para armazenar lucro
    all_net_rev = []
    ## dividindo em treino e validação
    target_train, target_valid, features_train, features_valid = train_test_split(target, features, train_size = 0.75, random_state = 1234)
    ## ajustando modelo
    lm = LinearRegression().fit(features_train, target_train)
    ## predizendo a produção
    prediction = pd.Series(lm.predict(features_valid))
    for i in range(1,1000):
        ## amostrando 500 locais com reposição e selecionando os 200 com maior produção prevista
        sample_loc = prediction.sample(500, replace= True).sort_values(ascending=False)[:200]
        ## calculando receita bruta da amostra
        gross_revenue = round(sum(sample_loc * 4500))
        ## calulando receita líquida
        net_revenue = gross_revenue - 100000000
        all_net_rev.append(net_revenue)
    return(all_net_rev)

**Comentário**: Defini uma função que estima o lucro líquido da produção de petróelo de uma região com poços. Para isso a função:
- separa as características e objetivo;
- divide as características e objetivo em 75% treino e 25% validação;
- ajusta um modelo linear com os dados de treino;
- prediz a produção de petróleo com as características de validação;
- itera 1000 vezes sobre as predições de produção e:
 - sorteia uma amosta aleatória com reposição de 500 predições de produção e seleciona as 200 mais produtivas;
 - calcula o lucro bruto com base na seleção de 200 predições mais produtivas;
 - calcula o lucro líquido ao descontar os custos de prospecção de 200 poços de petróleo;

In [22]:
def dist_revenue(rev):
    rev = pd.Series(rev)
    mean = rev.mean()
    q05 = rev.quantile(0.05)
    q95 = rev.quantile(0.95)
    return mean, q05, q95

**Comentário**: Defini uma função que descreve a distribuição do lucro líquido estimado. A função retorna a média e o quantiles de 5% e 95% da distribuição.


### Estimando lucro das diferentes regiões

In [23]:
### estimando lucro líquido das regiões
rev0 = project_revenue(geo = geo0)
rev1 = project_revenue(geo = geo1)
rev2 = project_revenue(geo = geo2)

In [ ]:
### descrevendo distribuição dos lucros estimados
mean_production = []
q05_production = []
q95_production = []
for rev in [rev0, rev1, rev2]:
    mean, q05, q95 =dist_revenue(rev = rev)
    mean_production.append(mean)
    q05_production.append(q05)
    q95_production.append(q95)

In [ ]:
### organizando descrições em um data frame
rev_df = pd.DataFrame({"region": region, "est_production": mean_production, "q05": q05_production, "q95": q95_production})
rev_df["CI"] = rev_df["q95"] - rev_df["q05"]
rev_df

**Comentário**: 
- A região 1 (`geo1`) teve o maior lucro líquido estimado, mas com o maior intervalo de confinaça sobre o lucro. Assim, esta região pode retornar tanto os maiores quanto os menores lucros, sendo a mais arriscada de investimento.
- A região 2 (`geo2`) teve o menor lucro líquido estimado, mas com o menor intervalo de confinça sobre o lucro. Assim, esta região é a mais segura em termos de risco, mas a menos lucrativa. 
- A região 0 (`geo0`) teve o lucro líquido estimado intermediário, assim como o intervalo de confiança sobre o lucro. Assim, esta região tem o melhor balanço entre lucro e segurança de investimento.

## Conclusão geral

Este projeto propôs escolher a melhor região para prospectar petróleo com base no lucro potencial de exploração da região. 
Foram comparadas três regiões cujos poços de petróleo atual tem características e produção conhecida. 

Com base nos dados, foram estimados a produção de petróleo e o lucro líquido que essas regiões poderiam retornar. O lucro líquido considerou os custos de prospecção de 200 poços. Tendo isto em vista, a região 0 (`geo`) mostrou o maior balanço entre lucro e risco potencial.